In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *
import signal, ast, concurrent.futures
from contextlib import contextmanager
import logging

# Configure logging
logging.basicConfig(filename='experiment.log', level=logging.INFO, format='%(asctime)s - %(message)s')


In [2]:
x = sp.symbols('x')
cap = 200 # total emission cap
Regulator8 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator8)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator8)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator8)
country1 = Country('DE', 1, regulator= Regulator8)
country2 = Country('FI', 0.5, regulator= Regulator8)
country3 = Country('GR', size= 0.1, regulator= Regulator8)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 10*x+ 2*x**2 + 0.1*x**3 , 0, 0, 0, regulator= Regulator8)
firm2 = Firm('firm2', 1, 2, x*0, 11*x+ 3*x**2 + 0.2*x**3, 0, 0, 0, regulator= Regulator8)
firm3 = Firm('firm3', 1, 3, x*0, 5*x+ 4*x**2 + 5*x**3 , 0, 0, 0, regulator= Regulator8)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator8)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator8)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator8)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator8)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator8)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator8)



In [3]:
# Function to run a method with a timeout
def run_with_timeout(method, timeout):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(method)
        try:
            future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print(f"Method {method.__name__} timed out.")
            logging.info(f"Method {method.__name__} timed out.")
        except Exception as e:
            print(f"Method {method.__name__} failed with exception: {e}")
            logging.info(f"Method {method.__name__} failed with exception: {e}")





: 

In [ ]:
# Load the data from the CSV file
df = pd.read_csv('../data/generated/exp04_Cap_vs_permit_price.csv')

# Initialize lists to store updated values
free_alloc = list(df['Free Allocation'])
caps2 = list(df['Emission Cap'])
permit_prices = list(df['Permit Price'])
outputs = [ast.literal_eval(output) for output in df['Outputs']]
emissions = [ast.literal_eval(emission) for emission in df['Emissions']]
profits = [ast.literal_eval(profit) for profit in df['Profits']]


# Iterate over each row in the DataFrame
for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    # Update the regulator and firms with data from the row
    Regulator8.emission_cap = row['Emission Cap']
    for firm, output, emission, profit in zip(Regulator8.firm_registry.values(), outputs[index], emissions[index], profits[index]):
        firm.actual_output = float(output)
        firm.emission = float(emission)

    for firm in Regulator8.firm_registry.values():
        firm.profit = firm.calculate_profit()
    # Run the equilibrium tester
    equilibrium_result, worst = Regulator8.equilibrium_tester(precision=1, full_output=False, output=True)
    print(f"Starting row {index} with emission cap {Regulator8.emission_cap} and old worst {worst}")
    # Store the best result
    best_result = {
        'Emission Cap': Regulator8.emission_cap,
        'Permit Price': Regulator8.permit_price,
        'Outputs': tuple([firm.actual_output for firm in Regulator8.firm_registry.values()]),
        'Emissions': tuple([firm.emission for firm in Regulator8.firm_registry.values()]),
        'Profits': tuple([firm.profit for firm in Regulator8.firm_registry.values()]),
        'Worst': worst
    }

    # If the result is not satisfactory, try other methods
    methods = [
        lambda: Regulator8.optimization_with_least_squares_ab(gurobi_print=False, print_output=False),
        lambda: Regulator8.optimization_with_positive_constraints_ab(gurobi_print=False, print_output=False),
        lambda: Regulator8.optimization_with_positive_constaints(gurobi_print=False, print_output=False),
        lambda: Regulator8.optimization_with_least_squares(gurobi_print=False, print_output=False),
        lambda: Regulator8.optimize_them_all()
    ]

    for i, method in enumerate(methods):
        print(f"Running method {i}")
        run_with_timeout(method, 60)  # Set timeout to 60 seconds
        equilibrium_result, new_worst = Regulator8.equilibrium_tester(precision=1, output=True)
        print(f"Method {i} finished with worst: {new_worst}")
        if new_worst < best_result['Worst']:
            logging.info(f"Using method {i}, the worst was updated from {best_result['Worst']} to {new_worst}.")
            best_result = {
                'Emission Cap': Regulator8.emission_cap,
                'Permit Price': Regulator8.permit_price,
                'Outputs': tuple([firm.actual_output for firm in Regulator8.firm_registry.values()]),
                'Emissions': tuple([firm.emission for firm in Regulator8.firm_registry.values()]),
                'Profits': tuple([firm.profit for firm in Regulator8.firm_registry.values()]),
                'Worst': new_worst
            }

    # Update the lists with the best values
    caps2[index] = best_result['Emission Cap']
    permit_prices[index] = best_result['Permit Price']
    outputs[index] = best_result['Outputs']
    emissions[index] = best_result['Emissions']
    profits[index] = best_result['Profits']

# Save the updated DataFrame to the CSV file
df = pd.DataFrame({
    'Free Allocation': free_alloc,
    'Emission Cap': caps2,
    'Permit Price': permit_prices,
    'Outputs': [str(output) for output in outputs],
    'Emissions': [str(emission) for emission in emissions],
    'Profits': [str(profit) for profit in profits]
})
df.to_csv('../data/generated/exp04_Cap_vs_permit_price.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |  +97.4138 ⚠️ | +143.8726 ⚠️ |  -10.9542 ✔️ |  -11.1542 ✔️ |   +2.1908 ✔️ | 3/5
Starting row 0 with emission cap 0.0 and old worst 143.872557882701
Running method 0
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Optimal solution found
Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      | +102.8873 ⚠️ |   +9.2131 ⚠️ |  -10.4484 ✔️ |  -10.6484 ✔️ |   +2.0897 ✔️ | 3/5
Method 0 finished with worst: 102.887291182726
Running method 1
Optimal solution found
Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------

  5%|▌         | 1/20 [00:47<14:55, 47.13s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +2.3561 ⚠️ |   +2.6665 ⚠️ |  -10.9549 ✔️ |  -11.1549 ✔️ |   +2.1910 ✔️ | 3/5
Method 4 finished with worst: 2.66645132427107
Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.9696 ✔️ |  +74.4750 ⚠️ |  -10.9543 ✔️ |  -11.1543 ✔️ |   +2.1909 ✔️ | 4/5
Starting row 1 with emission cap 117.32890163656192 and old worst 74.4750031620663
Running method 0
Optimal solution found
Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |  +65.5484 ⚠️ |   +6.0555 ⚠️ |  -10.6246 ✔️ |  -10.8246 ✔️ |   +2.1249 ✔️ | 3/5
Metho